In [1]:
import os
import shutil
import sys
import time
from PIL import Image
from datetime import date, datetime, timedelta


sys.path.insert(0, '..')
from utils import mongodb

In [2]:
from cocaina import cursor_cocaina, extract_to

## Generate cocaina database

#### Necessário acesso ao Servidor ajna.labin.rf08.srf (e habilitação no AJNA) ou acesso ao Servidor mongo.labin.rf08.srf (e senha do banco)

In [ ]:
#if os.path.exists('cocaina'):
#    shutil.rmtree('vazios')

In [3]:
start = datetime(2017, 7, 1)
end = datetime.today()
limit = 500
for label in (True, False):
    s0 = time.time()
    cursor = cursor_vazio_nvazio(mongodb, start, end, limit, label)
    count = extract_to(cursor)
    s1 = time.time()
    print('{:0.2f} segundos para processar {:d} registros'.format((s1 - s0), count))


(600, 365)  - abortando...
(600, 365)  - abortando...
(600, 357)  - abortando...
(25, 400)  - abortando...
(14, 400)  - abortando...
(600, 338)  - abortando...
(600, 350)  - abortando...
(600, 337)  - abortando...
(600, 352)  - abortando...
(600, 338)  - abortando...
(600, 352)  - abortando...
(600, 335)  - abortando...
(600, 350)  - abortando...
(512, 400)  - abortando...
(538, 400)  - abortando...
(600, 346)  - abortando...
(600, 349)  - abortando...
(600, 345)  - abortando...
(600, 375)  - abortando...
(600, 341)  - abortando...
(600, 376)  - abortando...
(600, 390)  - abortando...
(600, 387)  - abortando...
(600, 381)  - abortando...
(600, 338)  - abortando...
(600, 348)  - abortando...
(600, 338)  - abortando...
(600, 367)  - abortando...
(600, 334)  - abortando...
(600, 337)  - abortando...
(600, 342)  - abortando...
499 arquivos exportados...
8.79 segundos para processar 499 registros
(600, 353)  - abortando...
(600, 357)  - abortando...
(600, 343)  - abortando...
(600, 358)  - 

### Exportar pré processadas para visualização e cópia

#### Esta parte é para gerar arquivos em tamanho menor, possibilitando a cópia e compartilhamento (PenDrive, GoogleDrive, ReceitaDrive)

In [85]:
if not os.path.exists('transformed_vazios'):
    os.mkdir('transformed_vazios')
    os.mkdir('transformed_vazios/NVAZIO')
    os.mkdir('transformed_vazios/VAZIO')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
)

In [86]:
for classe in ('NVAZIO', 'VAZIO'):
    for i, batch in enumerate(train_datagen.flow_from_directory(
        'vazios',
        target_size=(144, 288),
        batch_size=10, 
        save_to_dir = 'transformed_vazios/' + classe,
        classes=[classe])):
        if i > 40:
            break

Found 480 images belonging to 1 classes.
Found 464 images belonging to 1 classes.


## Example: Training a very basic convolutional network

In [87]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [94]:
model = models.Sequential([
  layers.Conv2D(16, (5, 5), activation='relu',
                padding='same',
                input_shape=(144, 288, 3)),
  layers.MaxPooling2D(pool_size=(2, 4)),
  layers.Dropout(0.25),
  layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (1, 1), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(512, (1, 1), activation='relu'),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(1, activation='sigmoid')
 
])

model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_233 (Conv2D)          (None, 144, 288, 16)      1216      
_________________________________________________________________
max_pooling2d_104 (MaxPoolin (None, 72, 72, 16)        0         
_________________________________________________________________
dropout_127 (Dropout)        (None, 72, 72, 16)        0         
_________________________________________________________________
conv2d_234 (Conv2D)          (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_105 (MaxPoolin (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_128 (Dropout)        (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_235 (Conv2D)          (None, 34, 34, 64)      

In [95]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'vazios',
    target_size=(144, 288),
    batch_size=128,
    class_mode='binary',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'vazios',
    target_size=(144, 288),
    batch_size=60,
    class_mode='binary',
    subset='validation'
)


Found 756 images belonging to 2 classes.
Found 188 images belonging to 2 classes.


In [96]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                              verbose=1, min_delta=1e-2, mode='min')

model.fit_generator(train_generator, epochs=50,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 60)

Epoch 1/50
6/6 [==============================] - 22s 4s/step - loss: 0.6557 - acc: 0.5701 - val_loss: 0.6491 - val_acc: 0.5611
Epoch 2/50
6/6 [==============================] - 21s 3s/step - loss: 0.5515 - acc: 0.7672 - val_loss: 0.6218 - val_acc: 0.5111
Epoch 3/50
6/6 [==============================] - 20s 3s/step - loss: 0.4805 - acc: 0.7751 - val_loss: 0.4839 - val_acc: 0.8944
Epoch 4/50
6/6 [==============================] - 20s 3s/step - loss: 0.3947 - acc: 0.8452 - val_loss: 0.4763 - val_acc: 0.8389
Epoch 5/50
6/6 [==============================] - 21s 3s/step - loss: 0.3117 - acc: 0.8823 - val_loss: 0.4456 - val_acc: 0.8056
Epoch 6/50
6/6 [==============================] - 21s 3s/step - loss: 0.2502 - acc: 0.9127 - val_loss: 0.3247 - val_acc: 0.9333
Epoch 7/50
6/6 [==============================] - 20s 3s/step - loss: 0.3830 - acc: 0.8492 - val_loss: 0.3949 - val_acc: 0.8833
Epoch 8/50
6/6 [==============================] - 20s 3s/step - loss: 0.2130 - acc: 0.9259 - val_loss: 0

In [97]:
model.save_weights('gerabasesvazios.hdf5')

In [98]:
#Fine tunning
for layer in model.layers:
    layer.trainable = True

for layer in model.layers[:-7]:
    layer.trainable = False

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4,
                          verbose=1, min_delta=1e-2, mode='min')

model.compile(optimizer=optimizers.Adam(lr=0.000001), loss='binary_crossentropy', metrics=['acc'])
for l in model.layers:
    print(l.name, l.trainable)

conv2d_233 False
max_pooling2d_104 False
dropout_127 False
conv2d_234 False
max_pooling2d_105 False
dropout_128 False
conv2d_235 False
max_pooling2d_106 False
dropout_129 False
conv2d_236 False
conv2d_237 False
conv2d_238 False
max_pooling2d_107 False
dropout_130 False
conv2d_239 True
conv2d_240 True
conv2d_241 True
flatten_26 True
dense_52 True
dropout_131 True
dense_53 True


In [99]:
model.fit_generator(train_generator, epochs=20,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 88)

Epoch 1/20
6/6 [==============================] - 21s 4s/step - loss: 0.0596 - acc: 0.9802 - val_loss: 0.1052 - val_acc: 0.9833
Epoch 2/20
6/6 [==============================] - 19s 3s/step - loss: 0.0580 - acc: 0.9828 - val_loss: 0.1232 - val_acc: 0.9750
Epoch 3/20
6/6 [==============================] - 19s 3s/step - loss: 0.0607 - acc: 0.9802 - val_loss: 0.1129 - val_acc: 0.9833
Epoch 4/20
6/6 [==============================] - 19s 3s/step - loss: 0.0606 - acc: 0.9802 - val_loss: 0.1056 - val_acc: 0.9750
Epoch 5/20
5/6 [========================>.....] - ETA: 2s - loss: 0.0528 - acc: 0.9812
Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
6/6 [==============================] - 19s 3s/step - loss: 0.0631 - acc: 0.9788 - val_loss: 0.1132 - val_acc: 0.9750
Epoch 6/20
6/6 [==============================] - 20s 3s/step - loss: 0.0496 - acc: 0.9854 - val_loss: 0.1171 - val_acc: 0.9750
Epoch 7/20
6/6 [==============================] - 19s 3s/step - loss: 0.0622

In [100]:
model.save_weights('gerabasesvazios_tunned.hdf5')